## import packages

In [1]:
from pygeodyn.PYGEODYN import Pygeodyn
from pygeodyn.pygeodyn_plot import *

import datetime


# Run settings

In [2]:
run_list = [    'msis2',
                'tiegcm_oc',
                'ctipe_oc',
                'jb2008',
                'dtm2020_o',
                'gitm',
                'hasdm_oc',
           ]

plot_dir='/data/SatDragModelValidation/notebooks/Paper2023_icesat2_assessment/plots/'

dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    if i =='msis2':
        run_dict[i]={}
        run_dict[i]['num'] = 5
        run_dict[i]['model_path'] = None
        run_dict[i]['CD']         = 1.236791

    if i =='dtm2020_o':
        run_dict[i]={}
        run_dict[i]['num'] = 3
        run_dict[i]['model_path'] = None
        run_dict[i]['CD']         = 3.351462

    if i =='jb2008':
        run_dict[i]={}
        run_dict[i]['num'] = 1
        run_dict[i]['model_path'] = None
        run_dict[i]['CD']         = 1.908777
        
    ### PHYSICAL MODELS
    if i =='tiegcm_oc':
        run_dict[i]={}
        run_dict[i]['num'] = 0
        run_dict[i]['model_path'] = dir_modeldat+'/tiegcm/icesat2_oct2018_jan2019'
        run_dict[i]['CD']         = 1.372949
        
    if i =='ctipe_oc':
        run_dict[i]={}
        run_dict[i]['num']=4
        run_dict[i]['model_path'] = dir_modeldat+'/ctipe/icesat2_oct2018_jan2019'
        run_dict[i]['CD']         = 4.368334
        
    if i =='gitm':
        run_dict[i]={}
        run_dict[i]['num']=4
        run_dict[i]['model_path'] = dir_modeldat+'/gitm/icesat2_oct2018_jan2019'
        run_dict[i]['CD']         = 3.063949
        
    if i =='hasdm_oc':
        run_dict[i]={}
        run_dict[i]['num'] = 2
        run_dict[i]['model_path'] = dir_modeldat+'/hasdm/vishal_icesat2_oct2018_jan2019' #HASDM_OrbitCloud_2018313.01.csv
        run_dict[i]['CD']         = 2.076144


print(run_dict)

{'msis2': {'num': 5, 'model_path': None, 'CD': 1.236791}, 'tiegcm_oc': {'num': 0, 'model_path': '/data/SatDragModelValidation/data/inputs/atmos_models/tiegcm/icesat2_oct2018_jan2019', 'CD': 1.372949}, 'ctipe_oc': {'num': 4, 'model_path': '/data/SatDragModelValidation/data/inputs/atmos_models/ctipe/icesat2_oct2018_jan2019', 'CD': 4.368334}, 'jb2008': {'num': 1, 'model_path': None, 'CD': 1.908777}, 'dtm2020_o': {'num': 3, 'model_path': None, 'CD': 3.351462}, 'gitm': {'num': 4, 'model_path': '/data/SatDragModelValidation/data/inputs/atmos_models/gitm/icesat2_oct2018_jan2019', 'CD': 3.063949}, 'hasdm_oc': {'num': 2, 'model_path': '/data/SatDragModelValidation/data/inputs/atmos_models/hasdm/vishal_icesat2_oct2018_jan2019', 'CD': 2.076144}}


In [3]:
for i,den in enumerate(run_list):
    print(run_dict[den])

{'num': 5, 'model_path': None, 'CD': 1.236791}
{'num': 0, 'model_path': '/data/SatDragModelValidation/data/inputs/atmos_models/tiegcm/icesat2_oct2018_jan2019', 'CD': 1.372949}
{'num': 4, 'model_path': '/data/SatDragModelValidation/data/inputs/atmos_models/ctipe/icesat2_oct2018_jan2019', 'CD': 4.368334}
{'num': 1, 'model_path': None, 'CD': 1.908777}
{'num': 3, 'model_path': None, 'CD': 3.351462}
{'num': 4, 'model_path': '/data/SatDragModelValidation/data/inputs/atmos_models/gitm/icesat2_oct2018_jan2019', 'CD': 3.063949}
{'num': 2, 'model_path': '/data/SatDragModelValidation/data/inputs/atmos_models/hasdm/vishal_icesat2_oct2018_jan2019', 'CD': 2.076144}


In [ ]:
%load_ext autoreload
%autoreload 2
# from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

import pickle 
import os


file_raw_ICs = f"/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"\
          +"ICESat2_RawEphem_20181108_20181124.txt"


dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'icesat2/Paper2023_icesat2_assessment/results_2week_ScalingFactor/'

pickleName = '_2weeks_TwoWeekDragScaleFactor.pkl'

obj = {}
for i,den in enumerate(run_list):
    
    settings_icesat2= {# Basic input settings
                 'satellite'      : {'input': 'icesat2'},
                 'den_model'      : {'input': den},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'run_specifier'  : {'input': '_fixedscaledCD'},
                 'cd_model'       : {'input': 'BWDRAG'},
                 'file_string'    : {'input': 'CDmeanadj'},
                 'model_data_path' : {'input': run_dict[den]['model_path']},
                 # Force Model settings
                  'cd_type'               : {'input':"Fixed_CD"},
                  'cd_value'              : {'input':run_dict[den]['CD']},
                  'scaling_factor'        : {'input':False},
#                   'hours_between_cd_adj'  : {'input':24},
                  #### Comment for unadjusted run:
                  'cd_adjustment_boolean' : {'input':False },
                  #### ---------------------------------------
                 # Run
                  'step'           : {'input': 10.},
                  'orbfil_step'    : {'input': 120.},
                  'which_ICfile'   : {'input':file_raw_ICs},
                   #    # 312-328
                  'arc'            : {'input':[
                                               '2018.313',
                                               '2018.314',
#                                                '2018.315',
#                                                '2018.316',
#                                                '2018.317',
#                                                '2018.318',
#                                                '2018.319',
#                                                '2018.320',
#                                                '2018.321',
#                                                '2018.322', # failed convergence for MSIS
#                                                '2018.323',
#                                                '2018.324', # failed convergence for MSIS
#                                                '2018.325',
#                                                '2018.326',
                                              ]},
                  'epoch_start'    : {'input': [
                                                '2018-11-09 00:00:00',
                                                '2018-11-10 00:00:00',
#                                                 '2018-11-11 00:00:00',
#                                                 '2018-11-12 00:00:00',
#                                                 '2018-11-13 00:00:00',
#                                                 '2018-11-14 00:00:00',
#                                                 '2018-11-15 00:00:00',
#                                                 '2018-11-16 00:00:00',
#                                                 '2018-11-17 00:00:00',
#                                                 '2018-11-18 00:00:00',
#                                                 '2018-11-19 00:00:00',
#                                                 '2018-11-20 00:00:00',
#                                                 '2018-11-21 00:00:00',
#                                                 '2018-11-22 00:00:00',
                                               ]},
                   #
                  'epoch_stop'     : {'input':[ 
                                                '2018-11-10 00:00:00',
                                                '2018-11-11 00:00:00',
#                                                 '2018-11-12 00:00:00',
#                                                 '2018-11-13 00:00:00',
#                                                 '2018-11-14 00:00:00',
#                                                 '2018-11-15 00:00:00',
#                                                 '2018-11-16 00:00:00',
#                                                 '2018-11-17 00:00:00',
#                                                 '2018-11-18 00:00:00',
#                                                 '2018-11-19 00:00:00',
#                                                 '2018-11-20 00:00:00',
#                                                 '2018-11-21 00:00:00',
#                                                 '2018-11-22 00:00:00',
#                                                 '2018-11-23 00:00:00',
                                                ]},  
                   #

                                
                  'global_options' : {'input':'pso_2018'},
                 # Request read on raw outputs
                  'request_data'   : {'input': ['Trajectory_orbfil', 
                                               'Density', 
                                               'Residuals_summary',
                                               'RunSummary',
                                               'DragFile',
                                               'AdjustedParams'
                                               ]},
              #end dict
              }


    
    sat = Pygeodyn(settings_icesat2, use_file=False)
    sat.run_arcs()
#     obj[den] =  sat.getData_BigData_lowmemory()
    gc_collect()



#     pickle_file = dir_save+den+pickleName
#     if not os.path.exists(pickle_file):
#         print('Must create pickle file...')
#         print('   ',  pickle_file)
#         print('   ', 'Reading Geodyn Data')

#         ### Load the data into an object
#         sat = Pygeodyn(settings_icesat2, use_file=False)
#         obj = sat.getData_BigData_lowmemory()
#         gc_collect()

#         #### Pickle the object to save it
#         print('   ', 'Saving pickle')
#         filehandler = open(pickle_file, 'wb') 
#         pickle.dump(vars(obj), filehandler)
#         filehandler.close()
#         obj = 0
#         print('   ', 'Saved pickle')

# obj = {}
# for i,model in enumerate(run_list):     
#     ### Load the data if the pickles exist
#     print()
#     print()
#     gc_collect()

#     pickle_file = dir_save+model+pickleName

#     filehandler = open(pickle_file, 'rb') 
#     obj[model] = pickle.load(filehandler)
#     filehandler.close()
#     print('Loaded data from pickle... ', model)


    
    
# #     obj[den] =  sat.get_make_CleanData()
    
#     ### -----------------------------------------------------
# # startdate = "2018-11-08"
# # enddate   = "2018-11-24"  #24
# # sat = Pygeodyn(settings_icesat2, use_file=False)
# # sat.initialize_timeperiod_stage1(startdate, enddate,
# #                                  overwrite_exat=False, 
# #                                  overwrite_g2b=True)
# # import sys
# # sys.exit(0)

Using the ICESat-2 Class
Run #1     Current Time =      10:41:21  GMT-7
Run #1
|————————————————————————————————————————————————————————————————————————————————
| Run #1 Parameters
| —————————————————
|  Run Specs
|  --------- 
|    Satellite    icesat2
|    Run Type     DataReduction_PCE
|    CD Type      Fixed_CD
|    CD Value     1.236791
|    Density      msis2
|
|  Epoch Info 
|  ---------- 
|    Arc          2018.313
|    Arc length   24.0 hours
|    Epoch Start  2018-11-09 00:00:00
|    Epoch End    2018-11-10 00:00:00
|    Step Size    10.0 seconds
|
|  Files Info
|  ---------- 
|    Arc Name    icesat2_2018313.01_msis2.CDmeanadj
|
|    IISSET      .../data/inputs/sat_icesat2/setups/iisset.2018.313
|    EXAT        .../data/inputs/sat_icesat2/external_attitude/EXAT01.2018.313.gz
|    Output Raw  .../data/outputs_raw/icesat2/msis2/msis2_BWDRAG_fixedscaledCD/
|
|————————————————————————————————————————————————————————————————————————————————

Run #1          Running IIS
Run #1   

|     MSIS2 Density file already exists. 
|          - icesat2_2018314.01_msis2.CDmeanadj_msisin           
|     Already have OrbitCloud file for this arc: 
|          -OrbitCloud_icesat2_Step10_2018314.01.csv
|          - icesat2_2018314.01_ctipe_oc.CDmeanadj 
|     Running GEODYN with orbit cloud
Run #1     Current Time =      11:19:44  GMT-7
Run #1
|————————————————————————————————————————————————————————————————————————————————
| Run #1 Parameters
| —————————————————
|  Run Specs
|  --------- 
|    Satellite    icesat2
|    Run Type     DataReduction_PCE
|    CD Type      Fixed_CD
|    CD Value     4.368334
|    Density      ctipe_oc
|
|  Epoch Info 
|  ---------- 
|    Arc          2018.313
|    Arc length   24.0 hours
|    Epoch Start  2018-11-09 00:00:00
|    Epoch End    2018-11-10 00:00:00
|    Step Size    10.0 seconds
|
|  Files Info
|  ---------- 
|    Arc Name    icesat2_2018313.01_ctipe_oc.CDmeanadj
|
|    IISSET      .../data/inputs/sat_icesat2/setups/iisset.2018.313
| 


Run #1          Running IIS
Run #1          No errors in IIS
Run #1 ---------End of IIS

Run #1          Running IIE
Run #1          Current Time = 12:07:46 GMT-7
 ------ Current DIR:  /data/SatDragModelValidation/data/tmp/dtm2020_o_BWDRAG_fixedscaledCD/icesat2_2018313.01_dtm2020_o.CDmeanadj
Run #1          No errors in IIE
Run #1 ---------End of IIE
  - convergence achieved
Run #1          Time of IIE:  586.7491872310638 secs ( 9.77915312051773  mins)
Run #1          Current Time = 19:17:32
Saving fort.103 as drag_file
Saving fort.104 as SatGeometry_file
Run #1                Finished renaming files
Run #1                Finished copying files to outputdir
        Deleting tmp/:  /data/SatDragModelValidation/data/tmp/dtm2020_o_BWDRAG_fixedscaledCD/icesat2_2018313.01_dtm2020_o.CDmeanadj
Run #2     Current Time =      12:17:35  GMT-7
Run #2
|————————————————————————————————————————————————————————————————————————————————
| Run #2 Parameters
| —————————————————
|  Run Specs
|  ---------


Run #1          Running IIS
Run #1          No errors in IIS
Run #1 ---------End of IIS

Run #1          Running IIE
Run #1          Current Time = 12:40:08 GMT-7
 ------ Current DIR:  /data/SatDragModelValidation/data/tmp/hasdm_oc_BWDRAG_fixedscaledCD/icesat2_2018313.01_hasdm_oc.CDmeanadj
Run #1          No errors in IIE
Run #1 ---------End of IIE
  - convergence achieved
Run #1          Time of IIE:  401.9785780906677 secs ( 6.699642968177796  mins)
Run #1          Current Time = 19:46:50
Saving fort.103 as drag_file
Saving fort.104 as SatGeometry_file
Run #1                Finished renaming files
Run #1                Finished copying files to outputdir
        Deleting tmp/:  /data/SatDragModelValidation/data/tmp/hasdm_oc_BWDRAG_fixedscaledCD/icesat2_2018313.01_hasdm_oc.CDmeanadj
|     Running GEODYN with orbit cloud
Run #2     Current Time =      12:46:54  GMT-7
Run #2
|————————————————————————————————————————————————————————————————————————————————
| Run #2 Parameters
| ————————

1. Check if cleaned data exists  
   1. If not, process the `getData_BigData_lowmemory()`
   2. Further process the data into textfiles, saving the following.
       - One PCE file containing all arcs (XYZ, NTW, 120 sec cadence)
       - Seven Orbit Fit files for each density model (XYZ, NTW, 120 sec cadence)
       - Density File at proper time cadence (time, lon, lat, alt, to match orbfil data, 120 sec cadence)
       - Arc details file
           - Drag scaling factor
           - any other arc details
       

# Run output

## plot Drag acceleration scaling factor

In [ ]:
def calc_percent_change(a, b):
    from numpy import absolute as np_abs
        
    return(  ( (b-a)/np_abs(a) )*100)


In [ ]:
# %load_ext autoreload
# %autoreload 2


# def plot__ScalingFactor(fig, obj_m1, model_dict ):

#     cd_apriori  = 2.5

#     ####  Get plot Parameters for this model
#     model_m1 = obj_m1['global_params']['prms']['den_model']
#     col = get_plot_params( model_m1)
#     dateplot = []
#     rms_totals = []

#     fig.add_trace(go.Scattergl(x=model_dict['ScalingFactor_times'],
#                            y=model_dict['ScalingFactors'],
#                            name= model,
#                            mode='markers+lines',
#                            opacity=1,
#                                marker=dict(color=col, size=6 ),
#                            line = dict(shape = 'hvh', color = col, width=2),
#                            showlegend=False),
#                            secondary_y=False,row=1, col=1)

#     fig.add_trace(go.Scattergl(x=model_dict['ScalingFactor_times'],
#                            y=model_dict['percent_change'],
#                            name= model,
#                            mode='markers+lines',
#                            opacity=1,
#                                marker=dict(color=col, size=6 ),
#                            line = dict(shape = 'hvh', color = col, width=2),
#                            showlegend=False),
#                            secondary_y=False,row=2, col=1)

# # full_fig = fig.full_figure_for_development()

#     if model_m1 == 'jb2008':

#         fig.update_yaxes( title=r"$\text{Scaling Factor}$",
#                          range=[0.25,2.05], 
#                  exponentformat= 'power',row=1, col=1)
#         fig.update_yaxes( title=r"$\% \text{ change}$",
#                  exponentformat= 'power',row=2, col=1)
#         ###
#         ###  DATE on Final x-Axis only
#         fig.update_xaxes(title=r"$\text{Date}$", 
#                          range=[pd.to_datetime( "181108-160000", format='%y%m%d-%H%M%S'),
#                                 pd.to_datetime( "181123-120000", format='%y%m%d-%H%M%S')],
#                          row=2, col=1)


#     return(fig)





# ### -----------------------------------------------------------------------------------------------------
# ### -----------------------------------------------------------------------------------------------------
# ### -----------------------------------------------------------------------------------------------------
# ### -----------------------------------------------------------------------------------------------------
# ### -----------------------------------------------------------------------------------------------------

# ROWS_NUM = 2
# fig = make_subplots(rows=ROWS_NUM, cols=1,
#                     #                     
#                     shared_xaxes=True)

# # obj[den] = vars(obj[den])
# # satid = int(obj['jb2008']['global_params']['prms']['sat_ID'])
# satid = int(obj[den]['global_params']['prms']['sat_ID'])
# for plot_num, model in enumerate(run_list):
#     ScalingFactors      = []
#     ScalingFactor_times = []
#     PercChange          = []
#     for iarc,valarc in enumerate(obj[model]['global_params']['arc_input']):
#         arc = valarc+'.01'
#         iters = int(obj[model]['run_parameters'+arc]['total_iterations'])
#         for itime in obj[model]['AdjustedParams'][arc][iters][satid]['0CD'].keys():
#             CURRENT_VALUE = obj[model]['AdjustedParams'][arc][iters][satid]['0CD'][itime]['CURRENT_VALUE']
#             APRIORI_VALUE = obj[model]['AdjustedParams'][arc][iters][satid]['0CD'][itime]['APRIORI_VALUE']
#             ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
#             ScalingFactor_times.append(itime - datetime.timedelta(hours=12) )
#             PercChange.append(calc_percent_change(APRIORI_VALUE, CURRENT_VALUE))
            
#     run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
#     run_dict[model]['ScalingFactors']      = ScalingFactors

#     run_dict[model]['percent_change']      = PercChange

# for model in run_dict.keys():
#     fig = plot__ScalingFactor(fig, obj[model],  run_dict[model] )


# #######################################################
# font_dict=dict(family='Arial',size=16,color='black')
# #######################################################




# for i in [1,2]:
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=True,
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
#                       tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*2,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)


# modelnames=[]
# modelcolors = []
# #### LEGEND ####
# for model in run_dict.keys():
#     if model == 'msis2':
#         modelnames.append("MSISe2")
#         modelcolors.append(col_msis2)

#     elif model == 'dtm2020_o':
#         modelnames.append("DTM2020")
#         modelcolors.append(col_dtm2020)

#     elif model == 'jb2008':
#         modelnames.append("JB2008")
#         modelcolors.append(col_jb2008)

#     elif model == 'tiegcm_oc':
#         modelnames.append("TIEGCM")
#         modelcolors.append(col_tiegcm_oc)

#     elif model == 'hasdm_oc':
#         modelnames.append("HASDM")
#         modelcolors.append(col_hasdm_oc)

#     elif model == 'ctipe_oc':
#         modelnames.append("CTIPe")
#         modelcolors.append(col_ctipe_oc)

#     elif model == 'gitm':
#         modelnames.append("GITM")
#         modelcolors.append(col_gitm)



# df = pd.DataFrame({"starts_colors": modelcolors
#                                     })

# fig.update_traces(showlegend=False).add_traces(
#     [   go.Scattergl(name=modelnames[i], 
#                x=[pd.to_datetime( "181107-000000", format='%y%m%d-%H%M%S')],
#                mode='lines',
#                line = dict(shape = 'hv',  width=10),
#                marker_color=c, 
#                showlegend=True)
#         for i,c in enumerate((df.loc[:,["starts_colors"]].values.ravel()))])

# fig.update_layout(legend=dict(
#     yanchor="middle",
#     y=0.5,
#     xanchor="center",
#     x=1.1,
# #     x=1.015,
#         font=font_dict      ,
#         bgcolor="white",
#         bordercolor="darkgrey",
#         borderwidth=0.8,
#     )  )

# fig.update_layout(
# #                   title = '',
#                   autosize=False,    width=1000,    height=600,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict,
#                   plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=14)  # Increase size of subplot title

# fig.show(#renderer="jpg",
#          config=dict({
#             'displayModeBar': False,
#             'responsive': False,
#             'staticPlot': True,
#             'displaylogo': False,
#             'showTips': False,
#             }))

# #     if save_plot_flag:
# #         pio.write_image(fig, plot_dir+'Assessment_CDAvgAdj.jpg', scale=3)




In [ ]:
# obj[model]['AdjustedParams']['2018.313.01'][7][satid]['0CD'].keys()

### Scaling factor w Kp Cross correlation

In [ ]:
from netCDF4 import Dataset
def read_nc_file( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue.  '''
    status = os.path.exists(filename)
    
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )


arc_list = []

arc_list_18 = np.arange(292,366)
for i in arc_list_18:
    val = '2018'+str(i)
    arc_list.append(int(val))
    
    #     print(val)
    
arc_list_19 = np.arange(1,10)
for i in arc_list_19:
    val = '201900'+str(i)
    arc_list.append(int(val))

path =  "/data/SatDragModelValidation/data/inputs/atmos_models/geo_phys_indicies/"
path_to_f107 = path+ 'gpi_1960001-2021243_f107aDaily.nc'
variables = ['year_day', 'f107d', 'f107a', 'kp']
f107_data = read_nc_file(path_to_f107, variables)

date = []
kp_list = []
f107d_list = []
f107a_list  = []
date_3hr = []
doy_list    = []



for i,val in enumerate(arc_list):
    
    index = f107_data[0]['year_day']==val
    kp_list.append(f107_data[0]['kp'][index][0])
    f107d_list.append(f107_data[0]['f107d'][index][0])
    f107a_list.append(f107_data[0]['f107a'][index][0])
    doy_list.append(str(f107_data[0]['year_day'][index][0])[-3:])

    date.append(pd.to_datetime( str(val), format='%Y%j'))

    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=0))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=3))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=6))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=9))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=12))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=15))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=18))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=21))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=24))
    
kp_expand = []
for i in kp_list:
    for ii in i:
        kp_expand.append(ii)
        
        
        
solar_fluxes = {}
solar_fluxes['f107d_list'] = f107d_list
solar_fluxes['f107a_list'] = f107a_list
solar_fluxes['date']       = date
solar_fluxes['date_3hr']   = date_3hr
solar_fluxes['kp_expand']  = kp_expand

f107d_earth = []
f107a_earth = []
######################################################################### 
##### Account for the F10.7 at earth (instead of referenced at 1AU) #####
######################################################################### 

for i_doy,val_doy in enumerate(doy_list):
    iday = int(val_doy)
    theta0 = 2 * np.pi * (iday)/365.
    sfeps = 1.000110 + 0.034221*np.cos(theta0)+0.001280* np.sin(theta0) +0.000719*np.cos(2.*theta0)+0.000077*np.sin(2.*theta0)

    f107d_earth.append(sfeps * solar_fluxes['f107d_list'][i_doy])
    f107a_earth.append(sfeps * solar_fluxes['f107a_list'][i_doy])

solar_fluxes['f107d_earth'] = f107d_earth
solar_fluxes['f107a_earth'] = f107a_earth



### Prepare RMS total Plot arrays

arc_listlist=[  ['2018.292', '2018.293', '2018.294', '2018.295', '2018.296', 
                 '2018.297', '2018.298', '2018.299' ],                  
                #
                ['2018.304', '2018.305', '2018.306', '2018.307', '2018.308' ],  
                #
                ['2018.313', '2018.314', '2018.315', '2018.316', '2018.317',
                 '2018.318', '2018.319', '2018.320', '2018.321', '2018.322',
                 '2018.323', '2018.324', '2018.325', '2018.326', '2018.327' ],  
                #
                ['2018.335', '2018.336', '2018.337' ],  
                #
                ['2018.349', '2018.350', '2018.351', '2018.352' ],  
                #
                ['2018.356', '2018.357', '2018.358' ],  
                #
                ['2018.365', '2019.001', '2019.002', '2019.003', '2019.004', 
                 '2019.005', '2019.006', '2019.007', '2019.008',
                '2019.009'],  
                ]


In [ ]:
len(f107_data[0]['kp'])

In [ ]:
# len(kp_list)


date[21:36]

In [ ]:
fig = make_subplots(rows=2, cols=1,
                    vertical_spacing = 0.03,
                    shared_xaxes=True)


index1 = 21
index2 = 37
index3h_1 = 168
index3h_2 = 289

fig.add_trace(go.Scatter(x=solar_fluxes['date'][index1:index2],
                           y=solar_fluxes['f107d_earth'][index1:index2],
                           name= 'F107d_1AU',
                           mode='lines',
                           opacity=1,
#                                marker=dict(color='cornflowerblue', size=2 ),
                           line = dict(shape = 'hvh',dash='dash', color = 'black', width=2),
                           showlegend=False),
                           secondary_y=False,row=1, col=1)
fig.add_trace(go.Scatter(x=solar_fluxes['date'][index1:index2],
                           y=solar_fluxes['f107a_earth'][index1:index2],
                           name= 'F107a_1AU',
                           mode='lines',
                           opacity=1,
#                                marker=dict(color='cornflowerblue', size=2 ),
                           line = dict(shape = 'hvh', color = 'black', width=2),
                           showlegend=False),
                           secondary_y=False,row=1, col=1)

fig.add_trace(go.Scatter(x=solar_fluxes['date_3hr'][index3h_1:index3h_2],
                           y=solar_fluxes['kp_expand'][index3h_1:index3h_2],
                           name= 'Kp',
                           mode='lines',
                           opacity=1,
#                                marker=dict(color='black',size=2),
                           line = dict(shape = 'hvh', color = 'black', width=2),
                           showlegend=False),
                           secondary_y=False,row=2, col=1) 


index_1shaded=16
index3h_1shaded = 128
index3h_1shaded = 128

fig.add_trace(go.Scatter(x=solar_fluxes['date'][index_1shaded:index1+1],
                           y=solar_fluxes['f107d_earth'][index_1shaded:index1+1],
                           name= 'F107d',
                           mode='lines',
                           opacity=0.65,
                           line = dict(shape = 'hvh',dash='dash', color = 'black', width=1.5),
                           showlegend=False),
                           secondary_y=False,row=1, col=1)
fig.add_trace(go.Scatter(x=solar_fluxes['date'][index_1shaded:index1+1],
                           y=solar_fluxes['f107a_earth'][index_1shaded:index1+1],
                           name= 'F107a',
                           mode='lines',
                           opacity=0.65,
                           line = dict(shape = 'hvh', color = 'black', width=1.5),
                           showlegend=False),
                           secondary_y=False,row=1, col=1)

fig.add_trace(go.Scatter(x=solar_fluxes['date_3hr'][index3h_1shaded:index3h_1+1],
                           y=solar_fluxes['kp_expand'][index3h_1shaded:index3h_1+1],
                           name= 'Kp',
                           mode='lines',
                           opacity=0.55,
                           line = dict(shape = 'hvh', color = 'black', width=1.5),
                           showlegend=False),
                           secondary_y=False,row=2, col=1) 
### --------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(x=solar_fluxes['date'][index2-1:index2+5],
                           y=solar_fluxes['f107d_earth'][index2-1:index2+5],
                           name= 'F107d',
                           mode='lines',
                           opacity=0.65,
                           line = dict(shape = 'hvh',dash='dash', color = 'black', width=1.5),
                           showlegend=False),
                           secondary_y=False,row=1, col=1)
fig.add_trace(go.Scatter(x=solar_fluxes['date'][index2-1:index2+5],
                           y=solar_fluxes['f107a_earth'][index2-1:index2+5],
                           name= 'F107a',
                           mode='lines',
                           opacity=0.65,
                           line = dict(shape = 'hvh', color = 'black', width=1.5),
                           showlegend=False),
                           secondary_y=False,row=1, col=1)

fig.add_trace(go.Scatter(x=solar_fluxes['date_3hr'][index3h_2-1:index3h_2+40],
                           y=solar_fluxes['kp_expand'][index3h_2-1:index3h_2+40],
                           name= 'Kp',
                           mode='lines',
                           opacity=0.55,
                           line = dict(shape = 'hvh', color = 'black', width=1.5),
                           showlegend=False),
                           secondary_y=False,row=2, col=1) 



fig.add_vrect(x0=pd.to_datetime( str(2018308), format='%Y%j'),
              x1=pd.to_datetime( str(2018313), format='%Y%j'),
              fillcolor='gainsboro',
              opacity=.6,
              layer="below",
              line_width=0,
#               row=1,col=1,
             )
fig.add_vrect(x0=pd.to_datetime( str(2018328), format='%Y%j'),
              x1=pd.to_datetime( str(2018333), format='%Y%j'),
              fillcolor='gainsboro',
              opacity=.6,
              layer="below",
              line_width=0,
#               row=1,col=1,
             )


  
    
#######################################################
font_dict=dict(family='Arial',size=16,color='black')
#######################################################

fig.update_yaxes(title_text=r"$\text{F}_{\text{10.7}}\text{ Solar Flux (sfu)}$", 
#                  color='black',
                 range=[64.5, 86],
                  row=1, col=1,)

fig.update_yaxes(title_text=r"$\text{K}_\text{p}\text{ Index}$",
                  range=[-0.3, 8],
                 row=2, col=1,)

   

for i in [1,2]:
    if i ==1:
        xlabel=False
    else:
        xlabel=True
    
    fig.update_xaxes(### LINE at axis border
                      showline=True,showticklabels=xlabel,
                      linecolor='black',linewidth=1,
                     ### Major ticks
                      ticks='inside',tickfont=font_dict,mirror=True,
                      tickwidth=2,ticklen=9,
                      tickcolor='grey',tick0="2018-11-9" ,
                      dtick=86400000.0*2,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,ticklen=4,
                         tickcolor='grey',ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',gridwidth=1,
                       layer='above traces',tickangle=-45,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                     showticklabels=True,linecolor='black',  
                     linewidth=1,ticks='inside',tickfont=font_dict, 
                     mirror='allticks',tickwidth=1,tickcolor='black',
                     gridcolor='gainsboro',gridwidth=1,layer='above traces',
                     row=i, col=1)

fig.update_layout(autosize=False,    width=800,    height=600,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict, plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=14)  # Increase size of subplot title
fig.show(config=config)

### pio.write_image(fig, plots_dir+'twoweek_fullresult.pdf')
# pio.write_image(fig, plots_dir+'f107_kp_twoweek.jpg', scale=3)


In [ ]:
%load_ext autoreload
%autoreload 2


def plot__ScalingFactor(fig, obj_m1, model_dict ):

    cd_apriori  = 2.5

    ####  Get plot Parameters for this model
    model_m1 = obj_m1['global_params']['prms']['den_model']
    col = get_plot_params( model_m1)
    dateplot = []
    rms_totals = []


    fig.add_trace(go.Scattergl(x=model_dict['ScalingFactor_times'],
                           y=model_dict['percent_change'],
                           name= model,
                           mode='markers+lines',
                           opacity=1,
                               marker=dict(color=col, size=6 ),
                           line = dict(shape = 'hvh', color = col, width=2),
                           showlegend=False),
                           secondary_y=False,row=1, col=1)

# full_fig = fig.full_figure_for_development()

    if model_m1 == 'jb2008':

        fig.update_yaxes( title=r"$\text{Scaling Factor as } \% \text{ change}$",
                 exponentformat= 'power',row=1, col=1)
        ###
        ###  DATE on Final x-Axis only
#         fig.update_xaxes(title=r"$\text{Date}$", 
#                          range=[pd.to_datetime( "181108-160000", format='%y%m%d-%H%M%S'),
#                                 pd.to_datetime( "181123-120000", format='%y%m%d-%H%M%S')],
#                          row=1, col=1)


    return(fig)





### -----------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------

ROWS_NUM = 1
fig = make_subplots(rows=ROWS_NUM, cols=1,
                    #                     
                    shared_xaxes=True)

# obj[den] = vars(obj[den])
# satid = int(obj['jb2008']['global_params']['prms']['sat_ID'])
satid = int(obj[den]['global_params']['prms']['sat_ID'])
for plot_num, model in enumerate(run_list):
    ScalingFactors      = []
    ScalingFactor_times = []
    PercChange          = []
    for iarc,valarc in enumerate(obj[model]['global_params']['arc_input']):
        arc = valarc+'.01'
        iters = int(obj[model]['run_parameters'+arc]['total_iterations'])
        for itime in obj[model]['AdjustedParams'][arc][iters][satid]['0CD'].keys():
            CURRENT_VALUE = obj[model]['AdjustedParams'][arc][iters][satid]['0CD'][itime]['CURRENT_VALUE']
            APRIORI_VALUE = obj[model]['AdjustedParams'][arc][iters][satid]['0CD'][itime]['APRIORI_VALUE']
            ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
            ScalingFactor_times.append(itime - datetime.timedelta(hours=12) )
            PercChange.append(calc_percent_change(APRIORI_VALUE, CURRENT_VALUE))
            
    run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
    run_dict[model]['ScalingFactors']      = ScalingFactors

    run_dict[model]['percent_change']      = PercChange

for model in run_dict.keys():
    fig = plot__ScalingFactor(fig, obj[model],  run_dict[model] )


#######################################################
font_dict=dict(family='Arial',size=16,color='black')
#######################################################




for i in [1]:
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=True,
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
                      tickwidth=2,
                      ticklen=9,
                      tickcolor='grey',
                      tick0="2018-11-9" ,
                      dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       tickformat='%m/%d',
                    ticklabelstep=2,
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)



fig.update_layout(
#                   title = '',
                  autosize=False,    width=1000,    height=450,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=14)  # Increase size of subplot title

fig.show(#renderer="jpg",
         config=dict({
            'displayModeBar': False,
            'responsive': False,
            'staticPlot': True,
            'displaylogo': False,
            'showTips': False,
            }))

pio.write_image(fig, plot_dir+'24hourScalingFactors.jpg', scale=5)



In [ ]:
for model in run_dict.keys():
    print(f"{model}   {np.mean(run_dict[model]['percent_change'])}" )

In [ ]:
# run_dict['ctipe_oc']['percent_change']

In [ ]:
# solar_fluxes.keys()

In [ ]:
# solar_fluxes['date_3hr'][index3h_1:index3h_2-1]

In [ ]:
# solar_fluxes['date']

In [ ]:
run_dict[model]['CorrCoeff']=[]

In [ ]:
# index3h_1 = 168+4
# index3h_2 = 289-4

# solar_fluxes['kp_expand'][index3h_1:index3h_2]


# date[21:35]
# dailyKp

corrcoeffs = {}

for model in run_dict.keys():
    sf_kplen = []
    for sf in run_dict[model]['ScalingFactors']:
        sf_kplen.append( sf )

    print(model,'----------------')
    corr = np.corrcoef(sf_kplen, dailyKp ) 
    print(f"   R={corr[0,1]}")
    run_dict[model]['CorrCoeff'] = corr[0,1]

In [ ]:
# run_dict[model]['CorrCoeff']

In [ ]:
# date[21:35]
# dailyKp

In [ ]:
# path =  "/data/SatDragModelValidation/data/inputs/atmos_models/geo_phys_indicies/"
# # path_to_f107 = path+ 'gpi_1960001-2021243_f107aDaily.nc'
# # variables = ['year_day', 'f107d', 'f107a', 'kp']
# # f107_data = read_nc_file(path_to_f107, variables)


# '''
# Columns 1 to 4 
# Year of date of the UT day, for which data is given in this line

# MM 
# ii 
# Columns 6 to 7 
# Month of date of UT day

# DD
# ii
# Columns 9 to 10 
# Day of date of UT day

# days
# iiiii
# Columns 12 to 16
# Days since 1932-01-01 00:00 UT to start of UT day

# days_m
# fffff.f
# Columns 18 to 24
# Days since 1932-01-01 00:00 UT to midday of UT day

# BSR
# iiii
# Columns 26 to 29
# Bartels solar rotation number is a sequence of periods of exactly 27.0 UT days counted continuously from February 8, 1832

# dB
# ii
# Columns 31 to 32
# Day number within the Bartels solar rotation

# Kp (8 times)
# ii.iii (8 times)
# Columns 34 to 39, etc.
# Planetary three-hour index Kp for each of the intervals 00:00 to 03:00, 03:00 to 06:00, 06:00 to 09:00, 09:00 to 12:00, 12:00 to 15:00, 15:00 to 18:00, 18:00 to 21:00, 21:00 to 24:00 of the UT day, index values are rounded to three digits after the decimal point (one thousandth) and two digits are reserved for the index value before the decimal point to make the format consistent with that of the Hpo indices 

# ap (8 times)
# iiii (8 times)
# Columns 90 to 93, etc.
# Three-hourly equivalent planetary amplitude ap for each of the intervals 00:00 to 03:00, 03:00 to 06:00, 06:00 to 09:00, 09:00 to 12:00, 12:00 to 15:00, 15:00 to 18:00, 18:00 to 21:00, 21:00 to 24:00 of the UT day, four digits are reserved for ap to make the format consistent with that of the apo indices 

# Ap
# iiii
# Columns 132 to 134
# Daily equivalent planetary amplitude Ap, the arithmetic mean of the UT day's 8 ap values rounded to integer

# SN
# iii
# Columns 136 to 138
# International sunspot number SN

# F10.7obs
# ffffff.f
# Columns 140 to 147
# Noon-time observed solar radio flux F10.7 in s.f.u. (10^-22 W m^-2 Hz^-1)

# F10.7adj
# ffffff.f
# Columns 149 to 156
# Noon-time adjusted solar radio flux F10.7 in s.f.u. (10^-22 W m^-2 Hz^-1)

# D
# i
# Column 158
# D indicates if the Kp and SN values are definitive or preliminary. 
# D=0: Kp and SN preliminary
# D=1: Kp definitive, SN preliminary
# D=2 Kp and SN definitive


# '''



# pd.read_csv(path+'kp_2018.dat',
#             sep = '\s+',
#         names = ['YYYY',
#                 'MM',
#                 'DD',
#                 'days',
#                 'days_m',
#                 'BSR',
#                 'dB',
#                 'Kp',
#                 'Ap',
#                 'SN',
#                 'F10.7obs',
#                  'F10.7adj',
#                  'D',
#                                   ],

#            )

In [ ]:
dailyKp = []

for i in kp_list[21:37]:
    dailyKp.append(np.mean(i))

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=([None, "Correlation with Kp",
                                     None,  None ]),
                    vertical_spacing = 0.03,
                    horizontal_spacing = 0.02,
                        specs=[[{}, {}],
                               [{},  {}         ]],
                    column_widths=[0.8, 0.2],
                    shared_xaxes=True)

for model in run_dict.keys():
    fig = plot__ScalingFactor(fig, obj[model],  run_dict[model] )

    
    
fig.add_trace(go.Scatter(x=date[21:37],
                           y=dailyKp,
#                            name= 'F107d_1AU',
                           mode='lines',
                           opacity=1,
#                                marker=dict(color='cornflowerblue', size=2 ),
                           line = dict(shape = 'hv',dash='solid', color = 'black', width=3),
                           showlegend=False),
                           secondary_y=False,row=2, col=1)

fig.add_trace(go.Scatter(x=solar_fluxes['date_3hr'][index3h_1-5:index3h_2-5],
                           y=solar_fluxes['kp_expand'][index3h_1-5:index3h_2-5],
                           name= 'Kp',
                           mode='lines',
                           opacity=1,
#                                marker=dict(color='black',size=2),
                           line = dict(shape = 'hv',dash='dot', color = 'black', width=1.5),
                           showlegend=False),
                           secondary_y=False,row=2, col=1) 

fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="black", row=1, col=1)

#######################################################
font_dict=dict(family='Arial',size=16,color='black')
#######################################################

# fig.update_yaxes(title_text=r"$\text{F}_{\text{10.7}}\text{ Solar Flux (sfu)}$", 
# #                  color='black',
#                  range=[64.5, 86],
#                   row=1, col=1,)

# fig.update_yaxes(title_text=r"$\text{K}_\text{p}\text{ Index}$",
#                   range=[-0.3, 8],
#                  row=2, col=1,)

   

for i in [1,2]:
    if i ==1:
        xlabel=False
    else:
        xlabel=True
    
    fig.update_xaxes(### LINE at axis border
                      showline=True,showticklabels=xlabel,
                      linecolor='black',linewidth=1,
                     ### Major ticks
                      ticks='inside',tickfont=font_dict,mirror=True,
                      tickwidth=2,ticklen=9,
                      tickcolor='grey',tick0="2018-11-9" ,
                      dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       tickformat='%m/%d',
                    ticklabelstep=2,
                      #### Minor Ticks
                       minor=dict(dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,ticklen=4,
                         tickcolor='grey',ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',gridwidth=1,
                       layer='above traces',tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                     showticklabels=True,linecolor='black',  
                     linewidth=1,ticks='inside',tickfont=font_dict, 
                     mirror='allticks',tickwidth=1,tickcolor='black',
                     gridcolor='gainsboro',gridwidth=1,layer='above traces',
                     row=i, col=1)

fig.update_yaxes( title="Kp, daily average",
                 range=[-0.5,5],
                 exponentformat= 'power',row=2, col=1)



fig.add_trace(go.Bar(y=["JB2008",
                        "DTM2020",
                        "CTIPe",
                        "TIEGCM",
                        "MSIS2",
                        "GITM",
                        ],    
                     x=[run_dict["jb2008"]['CorrCoeff'],
                        run_dict["dtm2020_o"]['CorrCoeff'],
                        run_dict["ctipe_oc"]['CorrCoeff'],
                        run_dict["tiegcm_oc"]['CorrCoeff'],
                        run_dict["msis2"]['CorrCoeff'],
                        run_dict["gitm"]['CorrCoeff'],
                       ] ,
                     text=[np.round(run_dict["jb2008"]['CorrCoeff'],2)     ,
                           np.round(run_dict["dtm2020_o"]['CorrCoeff'],2)  ,
                           np.round(run_dict["ctipe_oc"]['CorrCoeff'],2)   ,
                           np.round(run_dict["tiegcm_oc"]['CorrCoeff'],2)  ,
                           np.round(run_dict["msis2"]['CorrCoeff'],2)      ,
                           np.round(run_dict["gitm"]['CorrCoeff'],2)       ,
                       ] ,
                    textposition='outside',
                     textfont=dict(family='Arial',size=13,color='black'),
                marker_color=all_cols,
                showlegend=False),row=1, col=2)
fig.update_traces(orientation="h", selector=dict(type='bar'))

fig.add_annotation(x=-0.5,y='GITM',
        xref="x",yref="y",
        showarrow=False,
        text='GITM',
        font=dict(size=20,color="black"),
        align="right",
        row=1, col=2,
        )
fig.add_annotation(x=-0.5,y='MSIS2',
        xref="x",yref="y",
        showarrow=False,
        text='MSIS2',
        font=dict(size=20,color="black"),
        align="right",
        row=1, col=2,
        )
fig.add_annotation(x=-0.5,y='TIEGCM',
        xref="x",yref="y",
        showarrow=False,
        text='TIEGCM',
        font=dict(size=20,color="black"),
        align="right",
        row=1, col=2,
        )
fig.add_annotation(x=-0.5,y='CTIPe',
        xref="x",yref="y",
        showarrow=False,
        text='CTIPe',
        font=dict(size=20,color="black"),
        align="right",
        row=1, col=2,
        )
fig.add_annotation(x=0.5,y='DTM2020',
        xref="x",yref="y",
        showarrow=False,
        text='DTM2020',
        font=dict(size=20,color="black"),
        align="left",
        row=1, col=2,
        )
fig.add_annotation(x=0.5,y='JB2008',
        xref="x",yref="y",
        showarrow=False,
        text='JB2008',
        font=dict(size=20,color="black"),
        align="left",
        row=1, col=2,
        )




fig.update_xaxes(### LINE at axis border
                  showline=True,showticklabels=True,
                  linecolor='gainsboro',linewidth=1,
                 ### Major ticks
                  ticks='inside',tickfont=dict(family='Arial',size=14,color='black'),mirror=True,
                  tickwidth=2,ticklen=9,
                  tickcolor='white',
                  ### GRID
                   gridcolor='white',gridwidth=1,
                   layer='above traces',
                    row=1, col=2)
fig.update_yaxes(showline=True,      # add line at x=0
                 showticklabels=False,linecolor='gainsboro',  
                 linewidth=1,ticks='inside',tickfont=dict(family='Arial',size=14,color='black'), 
                 mirror='allticks',tickwidth=1,tickcolor='gainsboro',
                 gridcolor='gainsboro',gridwidth=1,layer='above traces',
                                    row=1, col=2)

fig.update_xaxes(title="R, Pearson Corr. Coeff.",
                 range=[-1,1.05],
                 titlefont=dict(family='Arial',size=14,color='black'),
                 row=1, col=2)

fig.update_layout(autosize=False,    width=1000,    height=600,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict, plot_bgcolor='white', 
                 )
annot = fig['layout']['annotations'][0]
annot['yanchor']='bottom'
annot['y']=1.015
annot['yref']='paper'

fig.update_xaxes(range=[pd.to_datetime( "181109-000000", format='%y%m%d-%H%M%S'),
                        pd.to_datetime( "181123-000000", format='%y%m%d-%H%M%S')],
                 row=1, col=1)


fig.update_annotations(font_size=18)  # Increase size of subplot title
fig.show(config=config)

### pio.write_image(fig, plots_dir+'twoweek_fullresult.pdf')
# pio.write_image(fig, plots_dir+'f107_kp_twoweek.jpg', scale=3)

pio.write_image(fig, plot_dir+'24hourScalingFactors_w_R.jpg', scale=5)


In [ ]:
# Simplify Plotting Schemes:
col_msis2     =  "#2ca02c"  # 'tab:green'
col_jb2008    =  "#ff7f0e"  # 'tab:orange'
col_dtm2020   =  "#d62728"  # 'tab:red'
col_tiegcm_oc =  "#17becf"  # 'tab:cyan'
col_hasdm_oc  =  "#1f77b4"  # 'tab:blue'
col_ctipe_oc  =  "#9467bd"  # 'tab:purple'
col_gitm      =  '#e377c2'  # 'tab:pink'

all_cols = [col_jb2008,
            col_dtm2020,
            col_ctipe_oc,
            col_tiegcm_oc,
            col_msis2,
            col_gitm,
           ]


fig = go.Figure()
# for model in run_dict.keys():

fig.add_trace(go.Bar(y=["JB2008",
                        "DTM2020",
                        "CTIPe",
                        "TIEGCM",
                        "MSIS2",
                        "GITM",
                        ],    
                     x=[run_dict["jb2008"]['CorrCoeff'],
                        run_dict["dtm2020_o"]['CorrCoeff'],
                        run_dict["ctipe_oc"]['CorrCoeff'],
                        run_dict["tiegcm_oc"]['CorrCoeff'],
                        run_dict["msis2"]['CorrCoeff'],
                        run_dict["gitm"]['CorrCoeff'],
                       ] ,
                     text=[np.round(run_dict["jb2008"]['CorrCoeff'],2)     ,
                           np.round(run_dict["dtm2020_o"]['CorrCoeff'],2)  ,
                           np.round(run_dict["ctipe_oc"]['CorrCoeff'],2)   ,
                           np.round(run_dict["tiegcm_oc"]['CorrCoeff'],2)  ,
                           np.round(run_dict["msis2"]['CorrCoeff'],2)      ,
                           np.round(run_dict["gitm"]['CorrCoeff'],2)       ,
                       ] ,
            textposition='outside',
                marker_color=all_cols,
                ))
fig.update_traces(orientation="h", selector=dict(type='bar'))

fig.update_xaxes(### LINE at axis border
                  showline=True,showticklabels=True,
                  linecolor='black',linewidth=1,
                 ### Major ticks
                  ticks='inside',tickfont=font_dict,mirror=True,
                  tickwidth=2,ticklen=9,
                  tickcolor='grey',
                  ### GRID
                   gridcolor='gainsboro',gridwidth=1,
                   layer='above traces')
fig.update_yaxes(showline=True,      # add line at x=0
                 showticklabels=True,linecolor='black',  
                 linewidth=1,ticks='inside',tickfont=font_dict, 
                 mirror='allticks',tickwidth=1,tickcolor='black',
                 gridcolor='gainsboro',gridwidth=1,layer='above traces',)

fig.update_layout(autosize=True,   
                  legend= {'itemsizing': 'trace'},
                  font=font_dict, plot_bgcolor='white', 
                 )


fig.show()


## scaling and Kp correlation

In [ ]:
index3h_1 = 168+4
index3h_2 = 289-4

# solar_fluxes['kp_expand'][index3h_1:index3h_2]

print('Length of Kp list:', len(solar_fluxes['date_3hr'][index3h_1:index3h_2-1]))

solar_fluxes['date_3hr'][index3h_1:index3h_2-1]


for model in run_dict.keys():
    sf_kplen = []
    for sf in run_dict[model]['ScalingFactors']:
        sf_kplen.extend( np.squeeze(np.ones(8)*sf  ))

    print(model,'----------------')
    corr = np.corrcoef(sf_kplen, solar_fluxes['kp_expand'][index3h_1:index3h_2-1] ) 
    print(f"   R={corr[0,1]}")


In [ ]:
import sys
sys.exit(0)

In [ ]:
# import numpy as np
# from scipy import signal
# import matplotlib.pyplot as plt
# # rng = np.random.default_rng()



# sig1 = sf_kplen            #np.repeat([0., 1., 1., 0., 1., 0., 0., 1.], 128)
# sig2 = solar_fluxes['kp_expand'][index3h_1:index3h_2-1] 
# # sig_noise = sig + rng.standard_normal(len(sig))
# corr = signal.correlate(sig1, sig2, mode='same') 


# fig, (ax_orig, ax_noise, ax_corr) = plt.subplots(3, 1, sharex=True)
# ax_orig.plot(sig1)
# ax_orig.set_title('Scaling Factors for '+model)
# ax_noise.plot(sig2)
# ax_noise.set_title('Kp Signal')
# ax_corr.plot(corr)
# ax_corr.set_title('Cross-correlation')
# # ax_orig.margins(0, 0.1)
# fig.tight_layout()
# plt.show()



In [ ]:
# import matplotlib.pyplot as plt # matplot lib is the premiere plotting lib for Python: https://matplotlib.org/
# import numpy as np # numpy is the premiere signal handling library for Python: http://www.numpy.org/
# import scipy as sp # for signal processing
# from scipy import signal
# from scipy.spatial import distance
# # import IPython.display as ipd
# # import librosa
# # import random
# # import math
# # import makelab
# # # from makelab import audio
# # from makelab import signal

# def pad_zeros_right(s, padding_length):
#     # https://numpy.org/doc/1.18/reference/generated/numpy.pad.html
#     return np.pad(s, (0, padding_length), mode = 'constant', constant_values=0)

# def pad_mean_right(s, padding_length):
#     # https://numpy.org/doc/1.18/reference/generated/numpy.pad.html
#     return np.pad(s, (0, padding_length), mode = 'mean')

# # def compare_and_plot_signals(a, b, distance_function = distance.euclidean, alignment_function = None):
# def plot_signals_with_alignment(a, b, pad_function = None):
#     if(len(a) != len(b) and pad_function is None):
#         raise Exception(f"Signal 'a' and 'b' must be the same size; len(a)={len(a)} and len(b)={len(b)} or pad_function must not be None")
#     elif(len(a) != len(b) and pad_function is not None):
#         if(len(a) < len(b)):
#             a = pad_function(a, len(b) - len(a))
#         else:
#             b = pad_function(b, len(a) - len(b))
    
#     correlate_result = np.correlate(a, b, 'full')
#     shift_positions = np.arange(-len(a) + 1, len(b))
    
#     print("len(a)", len(a), "len(b)", len(b), "len(correlate_result)", len(correlate_result))

#     fig, axes = plt.subplots(5, 1, figsize=(10, 18))
    
#     axes[0].plot(a, alpha=0.7, label="a", marker="o")
#     axes[0].plot(b, alpha=0.7, label="b", marker="D")
#     axes[0].legend()
#     axes[0].set_title("Raw Signals 'a' and 'b'")
    
#     if len(shift_positions) < 20:
#         # useful for debugging and showing correlation results
#         print(shift_positions)
#         print(correlate_result)

#     best_correlation_index = np.argmax(correlate_result)
#     shift_amount_debug = shift_positions[best_correlation_index]
#     shift_amount = (-len(a) + 1) + best_correlation_index
#     print("best_correlation_index", best_correlation_index, "shift_amount_debug", shift_amount_debug, "shift_amount", shift_amount)
    
#     axes[1].stem(shift_positions, correlate_result, use_line_collection=True, label="Cross-correlation of a and b")
#     axes[1].set_title(f"Cross-Correlation Result | Best Match Index: {best_correlation_index} Signal 'b' Shift Amount: {shift_amount}")
#     axes[1].set_ylabel("Cross Correlation")
#     axes[1].set_xlabel("'b' Signal Shift Amount")
    
# #     best_match_ymin = 0
# #     best_match_ymin_normalized = makelab.signal.map(best_match_ymin, axes[1].get_ylim()[0], axes[1].get_ylim()[1], 0, 1)
# #     best_match_ymax = correlate_result[best_correlation_index]
# #     best_match_ymax_normalized = makelab.signal.map(best_match_ymax, axes[1].get_ylim()[0], axes[1].get_ylim()[1], 0, 1)
# #     axes[1].axvline(shift_positions[best_correlation_index], ymin=best_match_ymin_normalized, ymax=best_match_ymax_normalized, 
# #                     linewidth=2, color='orange', alpha=0.8, linestyle='-.', 
# #                     label=f"Best match ({shift_amount}, {best_match_ymax:.2f})")
# #     axes[1].legend()
    
# #     b_shifted_mean_fill = makelab.signal.shift_array(b, shift_amount, np.mean(b))
# #     axes[2].plot(a, alpha=0.7, label="a", marker="o")
# #     axes[2].plot(b_shifted_mean_fill, alpha=0.7, label="b_shifted_mean_fill", marker="D")
# #     axes[2].legend()
# #     axes[2].set_title("Signals 'a' and 'b_shifted_mean_fill'")
    
# #     b_shifted_zero_fill = makelab.signal.shift_array(b, shift_amount, 0)
# #     axes[3].plot(a, alpha=0.7, label="a", marker="o")
# #     axes[3].plot(b_shifted_zero_fill, alpha=0.7, label="b_shifted_zero_fill", marker="D")
# #     axes[3].legend()
# #     axes[3].set_title("Signals 'a' and 'b_shifted_zero_fill'")
    
#     b_shifted_roll = np.roll(b, shift_amount)
#     axes[4].plot(a, alpha=0.7, label="a", marker="o")
#     axes[4].plot(b_shifted_roll, alpha=0.7, label="b_shifted_roll", marker="D")
#     axes[4].legend()
#     axes[4].set_title("Signals 'a' and 'b_shifted_roll'")
    
#     fig.tight_layout()
    
# def compare_and_plot_signals_with_alignment(a, b, bshift_method = 'mean_fill', pad_function = None):
#     '''Aligns signals using cross correlation and then plots
    
#        bshift_method can be: 'mean_fill', 'zero_fill', 'roll', or 'all'. Defaults to 'mean_fill'
#     '''
    
#     if(len(a) != len(b) and pad_function is None):
#         raise Exception(f"Signal 'a' and 'b' must be the same size; len(a)={len(a)} and len(b)={len(b)} or pad_function must not be None")
#     elif(len(a) != len(b) and pad_function is not None):
#         if(len(a) < len(b)):
#             a = pad_function(a, len(b) - len(a))
#         else:
#             b = pad_function(b, len(a) - len(b))
    
#     correlate_result = np.correlate(a, b, 'full')
#     shift_positions = np.arange(-len(a) + 1, len(b))
#     print("len(a)", len(a), "len(b)", len(b), "len(correlate_result)", len(correlate_result))
    
#     euclid_distance_a_to_b = distance.euclidean(a, b)
    
#     num_charts = 3
#     chart_height = 3.6
#     if bshift_method is 'all':
#         num_charts = 5
    
#     fig, axes = plt.subplots(num_charts, 1, figsize=(10, num_charts * chart_height))
    
#     # Turn on markers only if < 50 points
#     a_marker = None
#     b_marker = None
#     if len(a) < 50:
#         a_marker = "o"
#         b_marker = "D"
        
#     axes[0].plot(a, alpha=0.7, label="a", marker=a_marker)
#     axes[0].plot(b, alpha=0.7, label="b", marker=b_marker)
#     axes[0].legend()
#     axes[0].set_title(f"Raw Signals | Euclidean Distance From 'a' to 'b' = {euclid_distance_a_to_b:.2f}")
    
#     if len(shift_positions) < 20:
#         # useful for debugging and showing correlation results
#         print(shift_positions)
#         print(correlate_result)
    
#     best_correlation_index = np.argmax(correlate_result)
#     shift_amount_debug = shift_positions[best_correlation_index]
#     shift_amount = (-len(a) + 1) + best_correlation_index
#     print("best_correlation_index", best_correlation_index, "shift_amount_debug", shift_amount_debug, "shift_amount", shift_amount)
    
#     #axes[1].plot(shift_positions, correlate_result)
#     axes[1].stem(shift_positions, correlate_result, use_line_collection=True, label="Cross-correlation of a and b")
#     axes[1].set_title(f"Cross-correlation result | Best match index: {best_correlation_index}; Signal 'b' shift amount: {shift_amount}")
#     axes[1].set_ylabel("Cross Correlation")
#     axes[1].set_xlabel("'b' Signal Shift Amount")
    
#     best_match_ymin = 0
# #     best_match_ymin_normalized = makelab.signal.map(best_match_ymin, axes[1].get_ylim()[0], axes[1].get_ylim()[1], 0, 1)
# #     best_match_ymax = correlate_result[best_correlation_index]
# #     best_match_ymax_normalized = makelab.signal.map(best_match_ymax, axes[1].get_ylim()[0], axes[1].get_ylim()[1], 0, 1)
# #     axes[1].axvline(shift_positions[best_correlation_index], ymin=best_match_ymin_normalized, ymax=best_match_ymax_normalized, 
# #                     linewidth=2, color='orange', alpha=0.8, linestyle='-.', 
# #                     label=f"Best match ({shift_amount}, {best_match_ymax:.2f})")
# #     axes[1].legend()
    
# #     if bshift_method is 'mean_fill' or bshift_method is 'all':
# #         b_shifted_mean_fill = makelab.signal.shift_array(b, shift_amount, np.mean(b))
# #         euclid_distance_a_to_b_shifted_mean_fill = distance.euclidean(a, b_shifted_mean_fill)
# #         axes[2].plot(a, alpha=0.7, label="a", marker=a_marker)
# #         axes[2].plot(b_shifted_mean_fill, alpha=0.7, label="b_shifted_mean_fill", marker=b_marker)
# #         axes[2].legend()
# #         axes[2].set_title(f"Euclidean distance From 'a' to 'b_shifted_mean_fill' = {euclid_distance_a_to_b_shifted_mean_fill:.2f}")
    
# #     ax_idx = 0
# #     if bshift_method is 'zero_fill' or bshift_method is 'all':
# #         if bshift_method is 'zero_fill':
# #             ax_idx = 2
# #         else:
# #             ax_idx = 3
    
# #         b_shifted_zero_fill = makelab.signal.shift_array(b, shift_amount, 0)
# #         euclid_distance_a_to_b_shifted_zero_fill = distance.euclidean(a, b_shifted_zero_fill)
# #         axes[ax_idx].plot(a, alpha=0.7, label="a", marker=a_marker)
# #         axes[ax_idx].plot(b_shifted_zero_fill, alpha=0.7, label="b_shifted_zero_fill", marker=b_marker)
# #         axes[ax_idx].legend()
# #         axes[ax_idx].set_title(f"Euclidean distance From 'a' to 'b_shifted_zero_fill' = {euclid_distance_a_to_b_shifted_zero_fill:.2f}")
    
    
#     if bshift_method is 'roll' or bshift_method is 'all':
#         if bshift_method is 'roll':
#             ax_idx = 2
#         else:
#             ax_idx = 4
#         b_shifted_roll = np.roll(b, shift_amount)
#         euclid_distance_a_to_b_shifted_roll = distance.euclidean(a, b_shifted_roll)
#         axes[ax_idx].plot(a, alpha=0.7, label="a", marker=a_marker)
#         axes[ax_idx].plot(b_shifted_roll, alpha=0.7, label="b_shifted_roll", marker=b_marker)
#         axes[ax_idx].legend()
#         axes[ax_idx].set_title(f"Euclidean distance From 'a' to 'b_shifted_roll' = {euclid_distance_a_to_b_shifted_roll:.2f}")
    
#     fig.tight_layout()
    


    
# compare_and_plot_signals_with_alignment(sig1, sig2)


## plot scaled results

In [ ]:
%load_ext autoreload
%autoreload 2


def plot_all_nokp(fig, obj_m1, model_dict ):

    cd_apriori  = 2.5

    ####  Get plot Parameters for this model
    model_m1 = obj_m1['global_params']['prms']['den_model']
    col = get_plot_params( model_m1)
    dateplot = []
    rms_totals = []

    
    for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
        arc =arc+'.01'
        dateplot.append(pd.to_datetime(datetime.datetime(int(arc.split('.')[0]), 1, 1) \
                                     + datetime.timedelta(int(arc.split('.')[1]))      \
                                     - datetime.timedelta(hours=12) ))
        rms_totals.append(obj_m1['Statistics'][arc]['T_RMS'].values[0])

        ### -----------------------------------------------------------------------------------------------------
        ###     DENSITY
        ###
        time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1['Density'], arc)

#         print('scaling factor',model_dict['ScalingFactors'][ii])
        

        
        ### -----------------------------------------------------------------------------------------------------
        ###     Orbit Averaged Density
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                    y=d_avg*(model_dict['ScalingFactors'][ii]),
#                                    ### name= model_m1,
#                                    mode='markers+lines',
#                                    opacity=1,
#                                        marker=dict(color=col,size=2),
#                                    ###     line = dict( color = col, width=2),
# #                                                    line = dict(shape='hvh', dash ='solid', color = col, width=2),
#                                        line = dict(dash ='solid', color = col, width=2),
#                                    showlegend=False), row=1, col=1)
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                y=d_avg,
#                                ### name= model_m1,
#                                mode='markers+lines',
#                                opacity=1,
#                                    marker=dict(color=col,size=2),
# #                                                line = dict( color = col, width=2),
# #                                    line = dict(shape='hvh', dash ='dot', color = col, width=2),
#                                showlegend=False), row=1, col=1)            


        ### -----------------------------------------------------------------------------------------------------
        ###     In Track Residuals
        data_resids = obj_m1['OrbitResids'][arc]['resids']
        fig.add_trace(go.Scattergl(x=data_resids['Date'][::50],
                                   y=data_resids['T'][::50],
                                   ###   name= model_m1,
                                     mode='markers+lines',
                                     opacity=1,
                                         marker=dict(color=col,size=2),
                                         line = dict( color = col, width=2),
                                     showlegend=False),
                                     secondary_y=False, row=1, col=1)
        ### -----------------------------------------------------------------------------------------------------
        ###     TOTAL RMS
#             print(rms_totals)
        if ii == 14:
#                     print(model_m1, "mean rms_totals:         ",np.round(np.mean(rms_totals),3) )
#                     print('       density scaling factor: ', np.round((cd_adjusted/cd_apriori),3))
            print(model_m1 )
            print("   mean rms_totals:       ",np.round(np.mean(rms_totals),3) )
            print('   density scaling factor:', np.round((cd_adjusted/cd_apriori),3))
        fig.add_trace(go.Scattergl(x=dateplot,
                                   y=rms_totals,
                                       ###e= 'NTW '+model_m1,
                                   mode='markers+lines',
                                         opacity=1,
                                         marker=dict(color=col,size=6),
                                         line = dict(shape='hvh', dash ='solid', color = col, width=2),
                                   showlegend=False),row=2, col=1)





# full_fig = fig.full_figure_for_development()

    if model_m1 == 'jb2008':
        ###
#         ### DENSITY AXIS
#         fig.update_yaxes(title_text=r"$\text{Orbit Avg. Density } (\frac{kg}{m^3})$", 
#                          type="log", 
# #                          range=yaxis2_range, 
#                          exponentformat= 'power',row=1, col=1)
        ###
        ### InTrack Residual Axis
        fig.update_yaxes( title=r"$\text{In-Track Residuals (m)}$",
#                          range=yaxis3_range, 
                         exponentformat= 'power',row=1, col=1)
        ###
        ### RMS AXIS
        fig.update_yaxes( title=r"$\text{In-Track RMSe (m)}$" ,type="linear" , exponentformat= 'power',
#                           range=yaxis4_range, 
                         row=2, col=1)
        ###
        ###  DATE on Final x-Axis only
        fig.update_xaxes(range=[pd.to_datetime( "181108-160000", format='%y%m%d-%H%M%S'),
                                pd.to_datetime( "181123-120000", format='%y%m%d-%H%M%S')],
                         row=2, col=1)


    return(fig)





### -----------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------


fig = make_subplots(rows=2, cols=1,
                    #                     
                    vertical_spacing = 0.03,
                    shared_xaxes=True)


satid = int(obj[den]['global_params']['prms']['sat_ID'])
for plot_num, model in enumerate(run_list):
    ScalingFactors  = []
    ScalingFactor_times = []
    for iarc,valarc in enumerate(obj[model]['global_params']['arc_input']):
        arc = valarc+'.01'
        iters = int(obj[model]['run_parameters'+arc]['total_iterations'])
        for itime in obj[model]['AdjustedParams'][arc][iters][satid]['0CD'].keys():
            CURRENT_VALUE = obj[model]['AdjustedParams'][arc][iters][satid]['0CD'][itime]['CURRENT_VALUE']
            APRIORI_VALUE = obj[model]['AdjustedParams'][arc][iters][satid]['0CD'][itime]['APRIORI_VALUE']
            ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
            ScalingFactor_times.append(itime)
    run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
    run_dict[model]['ScalingFactors']      = ScalingFactors


for model in run_dict.keys():
    fig = plot_all_nokp(fig, obj[model],  run_dict[model] )


#######################################################
font_dict=dict(family='Arial',size=16,color='black')
#######################################################




for i in [1,2]:
    if i==1:
        label_bool=False
    else:
        label_bool=True

    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=label_bool,
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
                      tickwidth=2,
                      ticklen=9,
                      tickcolor='grey',
                      tick0="2018-11-9" ,
                      dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       tickformat='%m/%d',
                    ticklabelstep=2,
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)



fig.update_layout(
#                   title = '',
                  autosize=False,    width=1000,    height=725,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
# fig.update_annotations(font_size=14)  # Increase size of subplot title

fig.show(#renderer="jpg",
         config=dict({
            'displayModeBar': False,
            'responsive': False,
            'staticPlot': True,
            'displaylogo': False,
            'showTips': False,
            }))

#     if save_plot_flag:
#         pio.write_image(fig, plot_dir+'Assessment_CDAvgAdj.jpg', scale=3)

pio.write_image(fig, plot_dir+'Fig_Assessment_24h_scaled.jpg', scale=5)

